<img src="./08/logo-insa.png" style="float:right; max-width: 60px; display: inline" alt="INSA" /></a>

# Sizing of the power converter on static criteria
*Written by Marc Budinger, INSA Toulouse, France*

The purpose of this notebook is to structure the sizing procedure of the inverter and its cooling system.

In general, the establishment of a sizing procedure involves the resolution of the following 3 problems:
* a set of equations sub-constrained by the addition of a design variable in the optimization problem;
* an over-constrained variable by adding a design variable (multiplier) and the transfer of the excess equation(s) in the constrained part of the optimization problem;
* an algebraic loop by the use of a simplify equation weighted by a multiplying coefficient and a constraint representing the initial equation.



Where possible:
* the design variables must take the form of a normalized variable around 1 (oversize coef. for example) or easily bounded to facilitate the work of the optimization algorithm.
* the constraints must take the form of inequality and not of equality more difficult to manage. The optimization of the objective (for example the total mass) will certainly force some (active) constraints to come to an end. 

## 1. Specification and equations

### 1.1. Inputs and specifications

The purpose of the sizing will be to minimize the mass of the cooling system. This mass depends mainly on the losses of the converter. In this notebook we will take into account the average value of losses in transistors and diodes.


In [5]:
Loss1D=91 # [W] Mean power loss of one diode of the inverter
Loss1T=139 # [W] Mean power loss of one transisotr of the inverter


### 1.2. Design assumptions

In [6]:
Nfan=1 # [-] Number of fans

## Thermal sizing scenario

### Design graph

> **Question:** Represent the sizing problem as an undirected graph. Use this graph to wisely choose the orientation of the calculations and the variables of the optimization problem.

![Graphic Thermal Design](08/IGBT_design_graph.png)

![NomColdplate](08/NomColdplate.png)
![Coldplate](08/Coldplate.png)
![Pipes](08/Pipes.png)
![Pump](08/Pump.png)
![Heatexchanger](08/Heatexchanger.jpg)
![Fan](08/Fan.png)


### Optimization problem: synthesis

> Exercice: synthesis here the design variables, the constraints which have to be add to the optimization problem.

![PseudoCode](08/PseudoCodeInductor.png)

![Procedure1](08/Procedure1.png)
![Procedure2](08/Procedure2.png)
![Procedure3](08/Procedure3.png)


### Sizing code and optimization

> **Exercice:** propose a sizing code and a optimization loop for the cooling system.

Before performing the overall optimization, each team member can test part of the code on a subset of the system in order to work collaboratively.

In [7]:
from math import pi, sqrt

# Coolants 
π = pi

'''Coolant flow characteristics (40°C)'''
λ_c  = 0.393              #[W/m.K] thermal conductivity
ρ_c  = 1030               #[kg/m3] density
cp_c = 3690               #[J/Kg.K] heat mass capacity
μ_c  = 0.00299            #[Pa.s] dynamic viscosity

'''Air characteristics (27°C)'''
T_in_a = 27            #[°C] input temperature
λ_a  = 0.0262          #[W/m.K] thermal conductivity
ρ_a  = 1.177           #[kg/m3] density 
cp_a = 1006            #[J/Kg.K] heat mass capacity
μ_a  = 1.85e-5         #[Pa.s] dynamic viscosity

T_air=40 # [K] Inlet air temperature

In [8]:
import numpy as np
import scipy
import scipy.optimize
import pandas as pd
import timeit
import ipywidgets as widgets
from ipywidgets import interactive

pd.options.display.float_format = '{:,.2f}'.format

def Sizing(param, arg):
    import math as m

    # Inputs
    # --

    T_j=param[0] # [K] Junction Temperature
    T_water_mean=param[1] # [K] Water temperature (cold)
    v_air=param[2] # [m/s] Inlet air speed (exchanger)
    Cr=param[3] # [-] Cr of Heat Exchenger, max=1
    NTU=param[4] # [-] NTU of Heat Exchanger, between 0 and 3
    RD_pipe=param[5] # [-] Ratio of pipe diameter
        
    # Power module
    # --

    Rth_jc_D=.15 # [K/W] Junction to case Diode Thermal resistance
    Rth_jc_T=.25 # [K/W] Junction to case Transistor Thermal resistance
    Rth_module = 0.036 # [K/W] Case to heatsink Thermal resistance of the module
    LossTotal = 6*(Loss1D+Loss1T) # Total Loss
    
    
    # Cold Plate
    # --

    T_ColdPlate = T_j - max(Rth_jc_D*Loss1D, Rth_jc_T*Loss1T) - Rth_module*LossTotal # Cold Plate temperature

    # Rth = C Q^Beta en [K/W] et Q en [l/min]
    β = -0.55
    C_Rth = 3.92E-02 

    Rth_ColdPlate=(T_ColdPlate-T_water_mean)/LossTotal # Target cold plate resistance
    Q_ColdPlate = (Rth_ColdPlate/C_Rth)**(1/β) # [l/min] 

    # Water temperature calculation (output of cold plate)
    T_water_cold = T_water_mean - LossTotal/(Q_ColdPlate*1e-3/60*cp_c*ρ_c)/2
    T_water_hot = T_water_mean + LossTotal/(Q_ColdPlate*1e-3/60*cp_c*ρ_c)/2

    # dP = C Q^Alpha
    ɣ = 1.47
    C_dP =  3.68E+02
    dP_ColdPlate=C_dP * Q_ColdPlate**ɣ  # [Pa]

    # Pipes
    # --

    # Reference
    
    L_pipes_ref   = 1.2          #[m] total length of the pipe
    D_pipes_ref   = 15.9e-3      #[m] pipes diameter
    dP_pipes_ref=105168 # [Pa] Pressure drop
    Q_pipes_ref=30 # [l/min] ref flow rate
    ML_pipes_ref=0.25 # [𝑘𝑔/𝑚] lineic mass
    
    # Pressure drop calculation
    dP_pipes = dP_pipes_ref*(Q_ColdPlate/Q_pipes_ref)**2*RD_pipe**(-4)
    
    Pipes_mass = ML_pipes_ref*RD_pipe*L_pipes_ref     
    
    # Heat exchanger
    # --
    
    HE_Eff=1-np.exp(NTU**0.22*(np.exp(-Cr*NTU**0.78)-1)/Cr) # [-] Heat exchanger efficacity
    
    # Airflow from Cr
    # Cr = Cmin / Cmax = Q_air.rho_air.c_air/Q_water.rho_water.c_water
    Q_water=Q_ColdPlate/1000/60 # [m^3/s]
    Cmax=Q_water*cp_c*ρ_c # [J/s]
    Cmin=Cr*Cmax # [J/s] 
    Q_air=Cmin/(cp_a*ρ_a) # [m^3/s] Air flow rate
    
    # Heat exchanger loss exchange possible
    LossHE=HE_Eff*(T_water_hot-T_air)*Cmin # [W] should be higher than LossTotal 
    
    # Surface and geometrical dimension
    # assumptions:
    # - n fans on the heat exchangers
    # -> L = NfanH
        
    # Surface from flowrate and air speed
    S_air=Q_air / v_air # [m^2] Surface of heat exchanger

    # As S_air = L H = Nfan H^2
    HE_H=m.sqrt(S_air /Nfan) # [m]
    HE_L=Nfan*HE_H # [m]
    
    # UA coefficient
    UA=NTU*Cmin 
    
    # Thickness 
    # The estimation function is: UA = 1.51e+04.L^1.41.W^1.05.H^0.51
    HE_W=(UA/1.51e4/HE_L**1.41/HE_H**0.51)**(1/1.05) # [m]
    
    # Heat exchanger mass
    
    HE_Mass=0.12*𝑈𝐴**0.793 # [kg]
    
    # Air Pressure drop
    HE_dP_air = 103.29*HE_L**(-2.34)*HE_W**(1.67)*HE_H**(-1.98)*Q_air**2 # [Pa]
    
    # Singular pressure drop in the car 
    # DP = ζ 1/2 ρ v^2

    ζ_global   = 4                      #[] ζ for bends, inlet ou outlet 
    
    dP_air_singular  = 0.5*ρ_a*v_air**2*ζ_global    #[Pa] pressure drop for all connections

    # Pump
    # --

    # Total pressure drop
    dP_water_total = dP_pipes + dP_ColdPlate  

    # Reference based on DC50B-12100
    Pump_Diam_ref=53 # [mm]
    Pump_Long_ref=100.3 # [mm]
    Pump_Mass_ref=.6 # [kg]
    Pump_DPmax_ref=1e5 # [Pa] max delta pressure
    Pump_DP_ref=.47*1e5 # [Pa] Rated delta pressure
    Pump_Q_ref=13 # [l/min] Rated Flow rate
    Pump_Qmax_ref=26 # [l/min] Rated Flow rate
    
    # Pump scaling
    # assumptions : 
    # - constant efficiency for the pump
    # - mass mainly due to the motor and thus the torque of the pump
    Pump_Mass=Pump_Mass_ref*((Q_ColdPlate/Pump_Q_ref)**(3/2)*(dP_water_total/Pump_DP_ref)**(1/4))**(3/3.5)
    
    Pump_DPmax=Pump_DPmax_ref*(dP_water_total/Pump_DP_ref) # [Pa]
    Pump_Qmax=Pump_Qmax_ref*(Q_ColdPlate/Pump_Q_ref)*1e-3/60 # [m^3/s]
 
    
    # Fan
    # --

    # reference based on SPAL VA99-BBL324P/N-101A/SH
                             
    Fan_D_ref = .280 # [m] Fan diameter
    Fan_T_ref = .077 # [m] Fan max thickness
    Fan_dP_max_ref = 480 # [Pa] Max Fan pressure drop (zero air flow rate) 
    Fan_dP_ref = 315 # [Pa] Fan pressure drop (max efficiency)
    Fan_Q_ref = 1500/3600 # [m^3/s] Fan air flow rate (max efficiency)
    Fan_Q_max_ref = 2600/3600 # [m^3/s] Fan air flow rate (zero pressure)
    Fan_W_ref = 2216*2*pi/60 # [rad/s] Fan speed 
    Fan_Mass_ref = 2 # [kg] mass

    # Fan Total pressure drop and flow rate
    Fan_dP  = dP_air_singular + HE_dP_air     #[Pa] total pressure losses
    Fan_Q = Q_air/Nfan # [m^3/s] deping of fan number fans

    # Other operating points (max)                            
    Fan_Q_max = Fan_Q_max_ref*Fan_Q/Fan_Q_ref # [m^3/s] 
    Fan_dP_max = Fan_dP_max_ref*Fan_dP/Fan_dP_ref # [Pa]
                             
    # Fan scaling
    # assumptions : 
    # - max speed limited by aerodynamic effect(Mach number)
    # - mass mainly due to the mechanical part of the fan
    
    Fan_Mass = Fan_Mass_ref*(Fan_Q/Fan_Q_ref)**(3/2)*(Fan_dP/Fan_dP_ref)**(1)
    
    # Aspect ratio of the fan
    Fan_ToverD=Fan_T_ref/Fan_D_ref*(Fan_dP/Fan_dP_ref)
    
    # Diameter
    Fan_D=Fan_D_ref*(Fan_Q/Fan_Q_ref)**(1/2)
    
    
    # Total mass (to optimize)
    # --
    Total_mass = Nfan*Fan_Mass + Pump_Mass + HE_Mass + Pipes_mass # [kg]

    # Objective and Constraints sum up
    # ---
    objective = Total_mass
    
    constraints = [(LossHE-LossTotal)/LossTotal, (T_ColdPlate-T_water_mean)/T_water_mean, (Fan_ToverD-Fan_T_ref/Fan_D_ref/2)/(Fan_T_ref/Fan_D_ref/2)]
   
    # Objective and contraints
    if arg=='Obj':
        return objective
    
    elif arg=='Prt':
    # the data to print a defined into a Pandas dataframe
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()


        df = df.append([{'Type': 'Specification', 'Name': 'Total loss', 'Value': LossTotal, 'Unit': '[W]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Objective', 'Name': 'Total mass', 'Value': objective, 'Unit': '[kg]', 'Comment': 'Global mass'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': '(LossHE-LossTotal)/LossTotal', 'Value': constraints[0], 'Unit': '[-]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': '(T_ColdPlate-T_water_mean)/T_water_mean', 'Value': constraints[1], 'Unit': '[-]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': '(Fan_ToverD-Fan_T_ref/Fan_D_ref/2)/(Fan_T_ref/Fan_D_ref/2)', 'Value': constraints[2], 'Unit': '[-]', 'Comment': ''}])[col_names]
        #df = df.append([{'Type': 'Constraints', 'Name': 'Const 3', 'Value': constraints[2], 'Unit': '[m]', 'Comment': 'Developped length'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Junction temperature', 'Value': param[0], 'Unit': '[K]', 'Comment': 'T_j'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Water temperature', 'Value': param[1], 'Unit': '[K]', 'Comment': 'T water mean'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Air speed', 'Value': param[2], 'Unit': '[m/s]', 'Comment': 'V air'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'Cr', 'Value': param[3], 'Unit': '[-]', 'Comment': 'HE'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'NTU', 'Value': param[4], 'Unit': '[-]', 'Comment': 'HE'}])[col_names]        
        df = df.append([{'Type': 'Variables', 'Name': 'RD_pipes', 'Value': param[5], 'Unit': '[-]', 'Comment': 'HE'}])[col_names]        
        df = df.append([{'Type': 'Pump', 'Name': 'Mass', 'Value': Pump_Mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Pump', 'Name': 'Pressure', 'Value': dP_water_total, 'Unit': '[Pa]', 'Comment': 'Pressure'}])[col_names]
        df = df.append([{'Type': 'Pump', 'Name': 'Flow Rate', 'Value': Q_ColdPlate, 'Unit': '[l/min]', 'Comment': 'Flow rate'}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Mass', 'Value': Fan_Mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Flow Rate', 'Value': Q_air*3600, 'Unit': '[m^3/h]', 'Comment': 'Flow rate'}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Delta Pressure', 'Value': Fan_dP , 'Unit': '[Pa]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Diameter', 'Value': Fan_D , 'Unit': '[m]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Fan', 'Name': 'Thickness', 'Value': Fan_D*Fan_ToverD , 'Unit': '[m]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Mass', 'Value': HE_Mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Efficacity', 'Value': HE_Eff, 'Unit': '[-]', 'Comment': ''}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'UA', 'Value': UA, 'Unit': '[-]', 'Comment': ''}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Surface', 'Value': S_air, 'Unit': '[m^2]', 'Comment': 'S_air'}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'L', 'Value': HE_L, 'Unit': '[m]', 'Comment': ''}])[col_names]        
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'H', 'Value': HE_H, 'Unit': '[m]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'W', 'Value': HE_W, 'Unit': '[m]', 'Comment': 'Thickness'}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Air speed', 'Value': v_air, 'Unit': '[m/s]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'T air in', 'Value': T_air, 'Unit': '[°C]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'T water in', 'Value': T_water_hot, 'Unit': '[°C]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'T water out', 'Value': T_water_cold, 'Unit': '[°C]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Air pressure drop', 'Value': HE_dP_air, 'Unit': '[Pa]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Heat Exchanger', 'Name': 'Air flow rate', 'Value': Q_air, 'Unit': '[m^3/s]', 'Comment': ''}])[col_names]    
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Flow rate', 'Value': Q_ColdPlate, 'Unit': '[l/min]', 'Comment': 'Flow rate'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Thermal resistance', 'Value': Rth_ColdPlate*1000, 'Unit': '[mK/W]', 'Comment': 'Rth_ColdPlate'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Temperature ColdPlate', 'Value': T_ColdPlate, 'Unit': '[°C]', 'Comment': 'T_ColdPlate'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Temperature Cold water', 'Value': T_water_cold, 'Unit': '[°C]', 'Comment': 'T_water_hot'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Temperature Hot water', 'Value': T_water_hot, 'Unit': '[°C]', 'Comment': 'T_water_hot'}])[col_names]
        df = df.append([{'Type': 'Cold Plate', 'Name': 'Pressure drop', 'Value': dP_ColdPlate, 'Unit': '[Pa]', 'Comment': ''}])[col_names]
        df = df.append([{'Type': 'Pipes', 'Name': 'Mass', 'Value': Pipes_mass, 'Unit': '[kg]', 'Comment': 'Mass'}])[col_names]
        df = df.append([{'Type': 'Pipes', 'Name': 'Diameter', 'Value': RD_pipe*D_pipes_ref*1e3 , 'Unit': '[mm]', 'Comment': 'Internal diameter'}])[col_names]
        df = df.append([{'Type': 'Pipes', 'Name': 'Pressure drop', 'Value': dP_pipes, 'Unit': '[Pa]', 'Comment': ''}])[col_names]
        
        # the dataframe is then organised to be printed with interactive widgets
        items = sorted(df['Type'].unique().tolist())
         
        #print(df.to_markdown()) 
        
        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
                  
    else:
        return constraints




In [9]:
#Variables d'optimisation

T_j=146 # [K] Junction Temperature
T_water_cold=55 # [K] Water temperature (cold)
v_air=5 # [m/s] Inlet air speed (exchanger)
Cr=0.9 # [-] Cr of Heat Exchenger, max=1
NTU=3 # [-] NTU of Heat Exchanger, between 0 and 3
RD_pipe=1 # [-] Diameter ratio compared to reference, around 1

# Vector of parameters
parameters = np.array((T_j,T_water_cold, v_air, Cr, NTU, RD_pipe))


In [10]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")

Sizing(parameters, 'Prt')
print("-----------------------------------------------")




,Type,Name,Value,Unit,Comment
0,Cold Plate,Flow rate,46.21,[l/min],Flow rate
0,Cold Plate,Thermal resistance,4.76,[mK/W],Rth_ColdPlate
0,Cold Plate,Temperature ColdPlate,61.57,[°C],T_ColdPlate
0,Cold Plate,Temperature Cold water,54.76,[°C],T_water_hot
0,Cold Plate,Temperature Hot water,55.24,[°C],T_water_hot
0,Cold Plate,Pressure drop,"103,037.74",[Pa],


-----------------------------------------------


In [11]:
# Then we can solve the problem and print of the optimized solution:

# In[70]:


# optimization with SLSQP algorithm
contrainte=lambda x: Sizing(x, 'Const')
objectif=lambda x: Sizing(x, 'Obj')

result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=[(100,150),(45,80),(1,15),(0.1,0.95),(0.1,3),(0.2,3)],
                                   f_ieqcons=contrainte, iter=100, acc=1e-8, epsilon=0.001)


# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

Sizing(result, 'Prt')

print("-----------------------------------------------")

,Type,Name,Value,Unit,Comment
0,Cold Plate,Flow rate,20.35,[l/min],Flow rate
0,Cold Plate,Thermal resistance,7.47,[mK/W],Rth_ColdPlate
0,Cold Plate,Temperature ColdPlate,65.57,[°C],T_ColdPlate
0,Cold Plate,Temperature Cold water,54.72,[°C],T_water_hot
0,Cold Plate,Temperature Hot water,55.79,[°C],T_water_hot
0,Cold Plate,Pressure drop,"30,861.37",[Pa],


-----------------------------------------------
